In [1]:
# """
# LINEAR REGRESSION MODEL - MATRIX COMPLETION IMPUTATION
# Complete equations for all hotels
# """

# import pandas as pd
# import numpy as np
# from pathlib import Path
# import json
# from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LassoCV
# import statsmodels.api as sm
# import warnings
# warnings.filterwarnings('ignore')

# def preprocess_dataset(df, y, hotel_id):
#     if hotel_id == 'FOCAL':
#         focal_lag_cols = [col for col in df.columns if 'base_rate_lag' in col.lower()]
#         df = df.drop(columns=focal_lag_cols)
#     else:
#         own_lag_cols = [col for col in df.columns if f'{hotel_id}_lag' in col]
#         df = df.drop(columns=own_lag_cols)
    
#     df['week_of_year'] = df['date'].dt.isocalendar().week.astype(float)
#     week_mean = df['week_of_year'].mean()
#     df['week_centered'] = df['week_of_year'] - week_mean
#     df['week_squared'] = df['week_centered'] ** 2
#     df['week_cubic'] = df['week_centered'] ** 3
#     df['week_quartic'] = df['week_centered'] ** 4
#     df['sin_week'] = np.sin(2 * np.pi * df['week_of_year'] / 52).astype(float)
#     df['cos_week'] = np.cos(2 * np.pi * df['week_of_year'] / 52).astype(float)
    
#     if 'is_holiday' not in df.columns:
#         df['is_holiday'] = 0
#         df.loc[df['date'].dt.month.isin([12, 1]), 'is_holiday'] = 1
#         df.loc[(df['date'].dt.month == 7) & (df['date'].dt.day <= 7), 'is_holiday'] = 1
#         df.loc[(df['date'].dt.month == 11) & (df['date'].dt.day >= 22), 'is_holiday'] = 1
    
#     df['is_peak_season'] = ((df['date'].dt.month.isin([6, 7, 8])) | 
#                             (df['date'].dt.month.isin([12, 1]))).astype(int)
#     df['is_summer'] = df['date'].dt.month.isin([6, 7, 8]).astype(int)
    
#     competitor_cols = [col for col in df.columns if 'booking-us' in col and 'lag' in col]
#     comp_corr_with_target = df[competitor_cols].corrwith(y).abs().sort_values(ascending=False)
    
#     all_competitors = []
#     for comp in comp_corr_with_target.index:
#         is_redundant = False
#         for existing in all_competitors:
#             if abs(df[comp].corr(df[existing])) > 0.90:
#                 is_redundant = True
#                 break
#         if not is_redundant:
#             all_competitors.append(comp)
    
#     seasonal_cols = ['is_holiday', 'is_peak_season', 'is_summer']
#     features_A = all_competitors + seasonal_cols + ['sin_week', 'cos_week']
#     features_B = all_competitors + seasonal_cols + ['week_centered', 'week_squared', 'week_cubic', 'week_quartic']
#     features_D = list(set(features_A + features_B))
    
#     return df, features_D

# def extract_equation_details(df, y, features_D, hotel_name):
#     X_D = df[features_D].dropna()
#     y_D = y[X_D.index]
    
#     scaler = StandardScaler()
#     X_D_scaled = scaler.fit_transform(X_D)
    
#     lasso = LassoCV(cv=5, random_state=42, max_iter=10000)
#     lasso.fit(X_D_scaled, y_D)
    
#     selected_mask = lasso.coef_ != 0
#     selected_features = np.array(features_D)[selected_mask]
    
#     if len(selected_features) == 0:
#         return {
#             'hotel': hotel_name,
#             'imputation_method': 'matrix_completion',
#             'intercept': float(y_D.mean()),
#             'coefficients': [],
#             'metrics': {
#                 'adj_r2': 0.0,
#                 'rmse': float(np.std(y_D)),
#                 'mape': float(np.mean(np.abs(y_D - y_D.mean()) / y_D) * 100),
#                 'n_features': 0,
#                 'n_observations': int(len(y_D)),
#                 'mean_price': float(np.mean(y_D))
#             }
#         }
    
#     X_selected = df.loc[X_D.index, list(selected_features)]
#     y_selected = y[X_D.index]
    
#     X_with_const = sm.add_constant(X_selected)
#     model = sm.OLS(y_selected, X_with_const).fit(cov_type='HC1')
    
#     coefficients = []
#     for feat in selected_features:
#         coef = model.params[feat]
#         pval = model.pvalues[feat]
        
#         if pval < 0.001:
#             sig = '***'
#         elif pval < 0.01:
#             sig = '**'
#         elif pval < 0.05:
#             sig = '*'
#         else:
#             sig = ''
        
#         if 'week' in feat.lower() or 'cos' in feat.lower() or 'sin' in feat.lower():
#             category = 'Temporal'
#         elif 'lag' in feat.lower():
#             category = 'Competitor'
#         else:
#             category = 'Seasonal'
        
#         coefficients.append({
#             'feature': feat,
#             'coefficient': float(coef),
#             'p_value': float(pval),
#             'significance': sig,
#             'category': category
#         })
    
#     y_pred = model.predict(X_with_const)
#     mape = float(np.mean(np.abs((y_selected - y_pred) / y_selected)) * 100)
    
#     return {
#         'hotel': hotel_name,
#         'imputation_method': 'matrix_completion',
#         'intercept': float(model.params['const']),
#         'coefficients': coefficients,
#         'metrics': {
#             'adj_r2': float(model.rsquared_adj),
#             'rmse': float(np.sqrt(model.mse_resid)),
#             'mape': mape,
#             'n_features': int(len(selected_features)),
#             'n_observations': int(len(y_selected)),
#             'mean_price': float(np.mean(y_selected))
#         }
#     }

# # Load data
# data_path = Path('../../../data/dataprocessed')
# df = pd.read_csv(data_path / 'lagged_predictive_dataset_matrix_completion.csv')
# df['date'] = pd.to_datetime(df['date'])
# df = df.sort_values('date').reset_index(drop=True)

# hotel_ids = [
#     'FOCAL',
#     'booking-us-aqua-pacific-monarch-USD',
#     'booking-us-castle-kamaole-sands-USD',
#     'booking-us-courtyard-by-marriott-maui-kahului-airport-USD',
#     'booking-us-kohea-kai-resort-maui-USD',
#     'booking-us-ohana-waikiki-malia-USD'
# ]

# all_equations = []

# for hotel_id in hotel_ids:
#     hotel_col = 'base_rate' if hotel_id == 'FOCAL' else hotel_id
#     if hotel_col not in df.columns:
#         continue
    
#     y = df[hotel_col].copy()
#     df_processed, features_D = preprocess_dataset(df.copy(), y, hotel_id)
#     equation = extract_equation_details(df_processed, y, features_D, hotel_id)
#     all_equations.append(equation)

# # Save
# output_path = Path('../../../data/dataprocessed')
# with open(output_path / 'model_equations_matrix_completion.json', 'w') as f:
#     json.dump(all_equations, f, indent=2)

# # Print all equations
# hotel_names_display = {
#     'FOCAL': 'FOCAL HOTEL',
#     'booking-us-aqua-pacific-monarch-USD': 'AQUA PACIFIC MONARCH',
#     'booking-us-castle-kamaole-sands-USD': 'CASTLE KAMAOLE SANDS',
#     'booking-us-courtyard-by-marriott-maui-kahului-airport-USD': 'COURTYARD MARRIOTT MAUI',
#     'booking-us-kohea-kai-resort-maui-USD': 'KOHEA KAI RESORT',
#     'booking-us-ohana-waikiki-malia-USD': 'OHANA WAIKIKI MALIA'
# }

# for eq in all_equations:
#     print(f"\n{'='*80}")
#     print(f"{hotel_names_display[eq['hotel']]} EQUATION - MATRIX COMPLETION")
#     print(f"{'='*80}")
#     print(f"\nPerformance: Adj R² = {eq['metrics']['adj_r2']:.4f}, MAPE = {eq['metrics']['mape']:.2f}%, RMSE = ${eq['metrics']['rmse']:.2f}")
    
#     target = 'base_rate' if eq['hotel'] == 'FOCAL' else 'price'
#     print(f"\nEquation:")
#     print(f"{target} = {eq['intercept']:.4f}")
    
#     temporal = [c for c in eq['coefficients'] if c['category'] == 'Temporal']
#     seasonal = [c for c in eq['coefficients'] if c['category'] == 'Seasonal']
#     competitor = [c for c in eq['coefficients'] if c['category'] == 'Competitor']
    
#     if temporal:
#         print(f"\n  # Temporal Features")
#         for c in temporal:
#             sign = '+' if c['coefficient'] > 0 else ''
#             sig = ' ' + c['significance'] if c['significance'] else ''
#             print(f"    {sign}{c['coefficient']:.6f} × {c['feature']}{sig}")
    
#     if seasonal:
#         print(f"\n  # Seasonal Features")
#         for c in seasonal:
#             sign = '+' if c['coefficient'] > 0 else ''
#             sig = ' ' + c['significance'] if c['significance'] else ''
#             print(f"    {sign}{c['coefficient']:.6f} × {c['feature']}{sig}")
    
#     if competitor:
#         print(f"\n  # Competitor Features")
#         for c in competitor:
#             sign = '+' if c['coefficient'] > 0 else ''
#             sig = ' ' + c['significance'] if c['significance'] else ''
#             print(f"    {sign}{c['coefficient']:.6f} × {c['feature']}{sig}")

# print(f"\n{'='*80}")
# print("Significance: *** p<0.001, ** p<0.01, * p<0.05")
# print(f"{'='*80}")

# # Summary table
# valid = [e for e in all_equations if e['metrics']['adj_r2'] > 0]
# avg_r2 = np.mean([e['metrics']['adj_r2'] for e in valid])
# avg_mape = np.mean([e['metrics']['mape'] for e in valid])
# avg_rmse = np.mean([e['metrics']['rmse'] for e in valid])

# print(f"\n{'='*80}")
# print("PERFORMANCE SUMMARY TABLE - MATRIX COMPLETION")
# print(f"{'='*80}\n")
# print(f"{'Hotel':<30} {'Adj. R²':<10} {'MAPE (%)':<10} {'RMSE ($)':<10} {'Features':<10}")
# print("-" * 70)
# for e in all_equations:
#     h = hotel_names_display[e['hotel']].title()
#     print(f"{h:<30} {e['metrics']['adj_r2']:<10.3f} {e['metrics']['mape']:<10.2f} {e['metrics']['rmse']:<10.2f} {e['metrics']['n_features']:<10}")
# print("-" * 70)
# print(f"{'Mean':<30} {avg_r2:<10.3f} {avg_mape:<10.2f} {avg_rmse:<10.2f}")

# print(f"\n{'='*80}")
# print("MATRIX COMPLETION SUMMARY:")
# print(f"  Working models: {len(valid)}/6")
# print(f"  Mean Adj R²: {avg_r2:.4f}")
# print(f"  Mean MAPE: {avg_mape:.2f}%")
# print(f"  Mean RMSE: ${avg_rmse:.2f}")
# print(f"  Correlation preserved: 99.0%")
# print("="*80)

In [2]:
"""
HYBRID IMPUTATION APPROACH
==========================
Matrix Completion for most hotels + Baseline fallback for failed models
"""

import pandas as pd
import numpy as np
from pathlib import Path
import json
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')

def preprocess_dataset(df, y, hotel_id):
    """Preprocess data and create features"""
    if hotel_id == 'FOCAL':
        focal_lag_cols = [col for col in df.columns if 'base_rate_lag' in col.lower()]
        df = df.drop(columns=focal_lag_cols, errors='ignore')
    else:
        own_lag_cols = [col for col in df.columns if f'{hotel_id}_lag' in col]
        df = df.drop(columns=own_lag_cols, errors='ignore')
    
    df['week_of_year'] = df['date'].dt.isocalendar().week.astype(float)
    week_mean = df['week_of_year'].mean()
    df['week_centered'] = df['week_of_year'] - week_mean
    df['week_squared'] = df['week_centered'] ** 2
    df['week_cubic'] = df['week_centered'] ** 3
    df['week_quartic'] = df['week_centered'] ** 4
    df['sin_week'] = np.sin(2 * np.pi * df['week_of_year'] / 52).astype(float)
    df['cos_week'] = np.cos(2 * np.pi * df['week_of_year'] / 52).astype(float)
    
    if 'is_holiday' not in df.columns:
        df['is_holiday'] = 0
        df.loc[df['date'].dt.month.isin([12, 1]), 'is_holiday'] = 1
        df.loc[(df['date'].dt.month == 7) & (df['date'].dt.day <= 7), 'is_holiday'] = 1
        df.loc[(df['date'].dt.month == 11) & (df['date'].dt.day >= 22), 'is_holiday'] = 1
    
    df['is_peak_season'] = ((df['date'].dt.month.isin([6, 7, 8])) | 
                            (df['date'].dt.month.isin([12, 1]))).astype(int)
    df['is_summer'] = df['date'].dt.month.isin([6, 7, 8]).astype(int)
    
    competitor_cols = [col for col in df.columns if 'booking-us' in col and 'lag' in col]
    comp_corr_with_target = df[competitor_cols].corrwith(y).abs().sort_values(ascending=False)
    
    all_competitors = []
    for comp in comp_corr_with_target.index:
        is_redundant = False
        for existing in all_competitors:
            if abs(df[comp].corr(df[existing])) > 0.90:
                is_redundant = True
                break
        if not is_redundant:
            all_competitors.append(comp)
    
    seasonal_cols = ['is_holiday', 'is_peak_season', 'is_summer']
    features_A = all_competitors + seasonal_cols + ['sin_week', 'cos_week']
    features_B = all_competitors + seasonal_cols + ['week_centered', 'week_squared', 'week_cubic', 'week_quartic']
    features_D = list(set(features_A + features_B))
    
    return df, features_D

def extract_equation(df, y, features_D, hotel_name, method_name):
    """Extract equation using Lasso + OLS"""
    X_D = df[features_D].dropna()
    y_D = y[X_D.index]
    
    scaler = StandardScaler()
    X_D_scaled = scaler.fit_transform(X_D)
    
    lasso = LassoCV(cv=5, random_state=42, max_iter=10000)
    lasso.fit(X_D_scaled, y_D)
    
    selected_mask = lasso.coef_ != 0
    selected_features = np.array(features_D)[selected_mask]
    
    if len(selected_features) == 0:
        return None  # Signal failure
    
    X_selected = df.loc[X_D.index, list(selected_features)]
    y_selected = y[X_D.index]
    
    X_with_const = sm.add_constant(X_selected)
    model = sm.OLS(y_selected, X_with_const).fit(cov_type='HC1')
    
    coefficients = []
    for feat in selected_features:
        coef = model.params[feat]
        pval = model.pvalues[feat]
        
        if pval < 0.001:
            sig = '***'
        elif pval < 0.01:
            sig = '**'
        elif pval < 0.05:
            sig = '*'
        else:
            sig = ''
        
        if 'week' in feat.lower() or 'cos' in feat.lower() or 'sin' in feat.lower():
            category = 'Temporal'
        elif 'lag' in feat.lower():
            category = 'Competitor'
        else:
            category = 'Seasonal'
        
        coefficients.append({
            'feature': feat,
            'coefficient': float(coef),
            'p_value': float(pval),
            'significance': sig,
            'category': category
        })
    
    y_pred = model.predict(X_with_const)
    mape = float(np.mean(np.abs((y_selected - y_pred) / y_selected)) * 100)
    
    return {
        'hotel': hotel_name,
        'imputation_method': method_name,
        'intercept': float(model.params['const']),
        'coefficients': coefficients,
        'metrics': {
            'adj_r2': float(model.rsquared_adj),
            'rmse': float(np.sqrt(model.mse_resid)),
            'mape': mape,
            'n_features': int(len(selected_features)),
            'n_observations': int(len(y_selected)),
            'mean_price': float(np.mean(y_selected))
        }
    }

print("="*80)
print("HYBRID IMPUTATION: MATRIX COMPLETION + BASELINE FALLBACK")
print("="*80)

# Paths for the datasets
matrix_path = Path('../../../data/dataprocessed')
baseline_path = Path('../../results/data/output-data')

# Load both datasets
print("\nLoading datasets...")
df_matrix = pd.read_csv(matrix_path / 'lagged_predictive_dataset_matrix_completion.csv')
df_matrix['date'] = pd.to_datetime(df_matrix['date'])
df_matrix = df_matrix.sort_values('date').reset_index(drop=True)
print(f"Matrix Completion data: {df_matrix.shape}")

df_baseline = pd.read_csv(baseline_path / 'lagged_predictive_dataset_baseline.csv')
df_baseline['date'] = pd.to_datetime(df_baseline['date'])
df_baseline = df_baseline.sort_values('date').reset_index(drop=True)
print(f"Baseline data: {df_baseline.shape}")

hotel_ids = [
    'FOCAL',
    'booking-us-aqua-pacific-monarch-USD',
    'booking-us-castle-kamaole-sands-USD',
    'booking-us-courtyard-by-marriott-maui-kahului-airport-USD',
    'booking-us-kohea-kai-resort-maui-USD',
    'booking-us-ohana-waikiki-malia-USD'
]

all_equations = []
failed_hotels = []

print("\n" + "="*80)
print("BUILDING MODELS")
print("="*80)

for hotel_id in hotel_ids:
    hotel_col = 'base_rate' if hotel_id == 'FOCAL' else hotel_id
    
    print(f"\nProcessing: {hotel_id}")
    
    # Try Matrix Completion first
    if hotel_col in df_matrix.columns:
        y = df_matrix[hotel_col].copy()
        df_processed, features_D = preprocess_dataset(df_matrix.copy(), y, hotel_id)
        equation = extract_equation(df_processed, y, features_D, hotel_id, 'matrix_completion')
        
        if equation is not None:
            all_equations.append(equation)
            print(f"  ✓ Matrix Completion - Adj R²: {equation['metrics']['adj_r2']:.4f}, MAPE: {equation['metrics']['mape']:.2f}%")
        else:
            print(f"  ✗ Matrix Completion FAILED - Trying baseline fallback...")
            failed_hotels.append(hotel_id)
            
            # Fallback to baseline
            if hotel_col in df_baseline.columns:
                y_base = df_baseline[hotel_col].copy()
                df_base_processed, features_D_base = preprocess_dataset(df_baseline.copy(), y_base, hotel_id)
                equation_base = extract_equation(df_base_processed, y_base, features_D_base, hotel_id, 'baseline_fallback')
                
                if equation_base is not None:
                    all_equations.append(equation_base)
                    print(f"  ✓ BASELINE Fallback - Adj R²: {equation_base['metrics']['adj_r2']:.4f}, MAPE: {equation_base['metrics']['mape']:.2f}%")
                else:
                    print(f"  ✗ BASELINE also failed!")
    else:
        print(f"  ✗ Column {hotel_col} not found")

# Save results
output_path = Path('../../../data/dataprocessed')
output_path.mkdir(parents=True, exist_ok=True)
with open(output_path / 'model_equations_hybrid.json', 'w') as f:
    json.dump(all_equations, f, indent=2)

# Print all equations
hotel_names_display = {
    'FOCAL': 'FOCAL HOTEL',
    'booking-us-aqua-pacific-monarch-USD': 'AQUA PACIFIC MONARCH',
    'booking-us-castle-kamaole-sands-USD': 'CASTLE KAMAOLE SANDS',
    'booking-us-courtyard-by-marriott-maui-kahului-airport-USD': 'COURTYARD MARRIOTT MAUI',
    'booking-us-kohea-kai-resort-maui-USD': 'KOHEA KAI RESORT',
    'booking-us-ohana-waikiki-malia-USD': 'OHANA WAIKIKI MALIA'
}

print("\n" + "="*80)
print("HYBRID MODEL EQUATIONS")
print("="*80)

for eq in all_equations:
    method_label = "MATRIX COMPLETION" if eq['imputation_method'] == 'matrix_completion' else "BASELINE FALLBACK"
    
    print(f"\n{'='*80}")
    print(f"{hotel_names_display[eq['hotel']]} - {method_label}")
    print(f"{'='*80}")
    print(f"\nPerformance: Adj R² = {eq['metrics']['adj_r2']:.4f}, MAPE = {eq['metrics']['mape']:.2f}%, RMSE = ${eq['metrics']['rmse']:.2f}")
    
    target = 'base_rate' if eq['hotel'] == 'FOCAL' else 'price'
    print(f"\nEquation:")
    print(f"{target} = {eq['intercept']:.4f}")
    
    temporal = [c for c in eq['coefficients'] if c['category'] == 'Temporal']
    seasonal = [c for c in eq['coefficients'] if c['category'] == 'Seasonal']
    competitor = [c for c in eq['coefficients'] if c['category'] == 'Competitor']
    
    if temporal:
        print(f"\n  # Temporal Features")
        for c in temporal:
            sign = '+' if c['coefficient'] > 0 else ''
            sig = ' ' + c['significance'] if c['significance'] else ''
            print(f"    {sign}{c['coefficient']:.6f} × {c['feature']}{sig}")
    
    if seasonal:
        print(f"\n  # Seasonal Features")
        for c in seasonal:
            sign = '+' if c['coefficient'] > 0 else ''
            sig = ' ' + c['significance'] if c['significance'] else ''
            print(f"    {sign}{c['coefficient']:.6f} × {c['feature']}{sig}")
    
    if competitor:
        print(f"\n  # Competitor Features")
        for c in competitor:
            sign = '+' if c['coefficient'] > 0 else ''
            sig = ' ' + c['significance'] if c['significance'] else ''
            print(f"    {sign}{c['coefficient']:.6f} × {c['feature']}{sig}")

print(f"\n{'='*80}")
print("Significance: *** p<0.001, ** p<0.01, * p<0.05")
print(f"{'='*80}")

# Summary table
valid = [e for e in all_equations if e['metrics']['adj_r2'] > 0]
avg_r2 = np.mean([e['metrics']['adj_r2'] for e in valid])
avg_mape = np.mean([e['metrics']['mape'] for e in valid])
avg_rmse = np.mean([e['metrics']['rmse'] for e in valid])

print(f"\n{'='*80}")
print("PERFORMANCE SUMMARY TABLE - HYBRID APPROACH")
print(f"{'='*80}\n")
print(f"{'Hotel':<35} {'Method':<20} {'Adj. R²':<10} {'MAPE (%)':<10} {'RMSE ($)':<10}")
print("-" * 85)
for e in all_equations:
    h = hotel_names_display[e['hotel']].title()
    m = "Matrix Comp." if e['imputation_method'] == 'matrix_completion' else "Baseline"
    print(f"{h:<35} {m:<20} {e['metrics']['adj_r2']:<10.3f} {e['metrics']['mape']:<10.2f} {e['metrics']['rmse']:<10.2f}")
print("-" * 85)
print(f"{'Mean':<35} {'':<20} {avg_r2:<10.3f} {avg_mape:<10.2f} {avg_rmse:<10.2f}")

print(f"\n{'='*80}")
print("HYBRID APPROACH SUMMARY:")
print(f"  Total models: {len(all_equations)}/6")
print(f"  Matrix Completion: {len([e for e in all_equations if e['imputation_method'] == 'matrix_completion'])}")
print(f"  Baseline Fallback: {len([e for e in all_equations if e['imputation_method'] == 'baseline_fallback'])}")
print(f"  Failed hotels: {failed_hotels if failed_hotels else 'None'}")
print(f"\n  Mean Adj R²: {avg_r2:.4f}")
print(f"  Mean MAPE: {avg_mape:.2f}%")
print(f"  Mean RMSE: ${avg_rmse:.2f}")
print("="*80)

print("\n✓ Saved: model_equations_hybrid.json")

HYBRID IMPUTATION: MATRIX COMPLETION + BASELINE FALLBACK

Loading datasets...
Matrix Completion data: (359, 53)
Baseline data: (360, 68)

BUILDING MODELS

Processing: FOCAL
  ✓ Matrix Completion - Adj R²: 0.6192, MAPE: 6.19%

Processing: booking-us-aqua-pacific-monarch-USD
  ✓ Matrix Completion - Adj R²: 0.7250, MAPE: 4.51%

Processing: booking-us-castle-kamaole-sands-USD
  ✓ Matrix Completion - Adj R²: 0.5340, MAPE: 9.07%

Processing: booking-us-courtyard-by-marriott-maui-kahului-airport-USD
  ✓ Matrix Completion - Adj R²: 0.6851, MAPE: 4.00%

Processing: booking-us-kohea-kai-resort-maui-USD
  ✗ Matrix Completion FAILED - Trying baseline fallback...
  ✓ BASELINE Fallback - Adj R²: 0.5596, MAPE: 2.89%

Processing: booking-us-ohana-waikiki-malia-USD
  ✓ Matrix Completion - Adj R²: 0.7619, MAPE: 5.83%

HYBRID MODEL EQUATIONS

FOCAL HOTEL - MATRIX COMPLETION

Performance: Adj R² = 0.6192, MAPE = 6.19%, RMSE = $20.91

Equation:
base_rate = 111.7659

  # Temporal Features
    -0.000656 × we